In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from skimage.feature import hog

In [2]:
def intersection_over_union(coord, coord_label):
    """
    coord : (x_haut_gauche, y_haut_gauche, x_bas_droite, y_bas_droite)
    coord_label : (x_haut_gauche, y_haut_gauche, x_bas_droite, y_bas_droite)
    """

    x1, y1, x2, y2 = coord
    x1_l, y1_l, x2_l, y2_l = coord_label

    x_inter_1 = max(x1, x1_l)
    y_inter_1 = max(y1, y1_l)
    x_inter_2 = min(x2, x2_l)
    y_inter_2 = min(y2, y2_l)

    interArea = max(0, x_inter_2 - x_inter_1 + 1) * max(0, y_inter_2 - y_inter_1 + 1)

    boxArea = (x2 - x1 + 1) * (y2 - y1 + 1)
    boxArea_label = (x2_l - x1_l + 1) * (y2_l - y1_l + 1)

    return interArea / (boxArea + boxArea_label - interArea)

In [3]:
train_img = os.path.join("dataset", "train", "images")
train_lab = os.path.join("dataset", "train", "labels")
train_list = os.listdir(train_img)

val_img = os.path.join("dataset", "val", "images")
val_lab = os.path.join("dataset", "val", "labels")
val_list = os.listdir(val_img)

test_img = os.path.join("dataset", "test")
test_list = os.listdir(test_img)

In [4]:
path_new_dataset = os.path.join("full_dataset")
    
subfolders = ["frouge", "fvert", "forange", "interdiction", "danger", "stop", "ceder", "obligation", "none"]

results = {
    "frouge": 0,
    "fvert": 0,
    "forange": 0,
    "interdiction": 0,
    "danger": 0,
    "stop": 0,
    "ceder": 0,
    "obligation": 0,
    "none": 0
}

for subfolder in subfolders:
    results[subfolder] = os.path.join(path_new_dataset, subfolder)

## Entrainement

In [5]:
# On ajoute toutes les images des dossiers dans un dataframe
X_train_detector = []
Y_train_detector = []

X_train_classifier = []
Y_train_classifier = []

correspondance = {
    "none": 0,
    "frouge": 1,
    "fvert": 2,
    "forange": 3,
    "interdiction": 4,
    "danger": 5,
    "stop": 6,
    "ceder": 7,
    "obligation": 8
}

taille = 32

for subfolder in subfolders:
    for img in tqdm(os.listdir(os.path.join(path_new_dataset, subfolder))):
        img_data = cv2.imread(os.path.join(path_new_dataset, subfolder, img))
        img_data = cv2.resize(img_data, (taille, taille), interpolation=cv2.INTER_AREA)
        hog_img = hog(cv2.cvtColor(img_data, cv2.COLOR_BGR2GRAY), orientations=9, pixels_per_cell=(2, 2), visualize=False)

        X_train_detector.insert(0, hog_img)
        Y_train_detector.insert(0, 0 if subfolder == "none" else 1)
        
        hist_red = cv2.calcHist([img_data], [0], None, [256], [0, 256])
        hist_red = hist_red / hist_red.sum()
        hist_green = cv2.calcHist([img_data], [1], None, [256], [0, 256])
        hist_green = hist_green / hist_green.sum()
        hist_blue = cv2.calcHist([img_data], [2], None, [256], [0, 256])
        hist_blue = hist_blue / hist_blue.sum()
        
        X_train_classifier.insert(0, np.concatenate((hog_img, hist_red.flatten(), hist_green.flatten(), hist_blue.flatten())))
        Y_train_classifier.insert(0, correspondance[subfolder])

100%|██████████| 3792/3792 [00:39<00:00, 96.86it/s] 


In [6]:
X_train_detector = np.array(X_train_detector)
Y_train_detector = np.array(Y_train_detector)

In [7]:
X_train_classifier = np.array(X_train_classifier)
Y_train_classifier = np.array(Y_train_classifier)   

In [8]:
from sklearn.ensemble import RandomForestClassifier

model_detector = RandomForestClassifier(n_jobs=-1, class_weight="balanced")
model_detector.fit(X_train_detector, Y_train_detector)

RandomForestClassifier(class_weight='balanced', n_jobs=-1)

In [9]:
model_classifier = RandomForestClassifier(n_jobs=-1, class_weight="balanced")
model_classifier.fit(X_train_classifier, Y_train_classifier)

RandomForestClassifier(class_weight='balanced', n_jobs=-1)

Enregistrement des modèles

In [ ]:
import joblib

filename_detector = 'detector_32_32_hog.sav'
joblib.dump(model_detector, filename_detector)

filename_classifier = 'classifier_32_32_hog_hist.sav'
joblib.dump(model_classifier, filename_classifier)

## Récupération des modèles et prédiction

In [ ]:
import joblib

# Load the saved models
filename_detector = 'detector_32_32_hog.sav'
model_detector = joblib.load(filename_detector)

filename_classifier = 'classifier_32_32_hog_hist.sav'
model_classifier = joblib.load(filename_classifier)

In [10]:
def is_image_too_blue(img, blue_threshold=1.2):
    """
    Vérifie si une image est trop bleue.
    """
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    r, g, b = cv2.split(img_rgb)
    mean_r = np.mean(r)
    mean_g = np.mean(g)
    mean_b = np.mean(b)
    return (mean_b / ((mean_r + mean_g) / 2)) > blue_threshold  

def detect_ROI(img_path, erode=0, dilate=0, opening=0, closing=0, both=0):
    """
    Transforme image to HSV, mask on red, green, orange, blue, make erosion, dilatation on each mask, find include rectangle
    return all rectangle
    """
    img = cv2.imread(img_path)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    Bmin = np.array([85, 43, 46])
    Bmax = np.array([140, 255, 255])
    img_Bbin = cv2.inRange(img_hsv,Bmin, Bmax)
    
    list_mask = [img_Bbin]
    
        
    Rmin1 = np.array([0, 43, 46])
    Rmax1 = np.array([10, 255, 255])
    img_Rbin1 = cv2.inRange(img_hsv,Rmin1, Rmax1)
    
    Rmin2 = np.array([156, 43, 46])
    Rmax2 = np.array([180, 255, 255])
    img_Rbin2 = cv2.inRange(img_hsv,Rmin2, Rmax2)
    img_Rbin = np.maximum(img_Rbin1, img_Rbin2)
    list_mask.append(img_Rbin)

    
    # Detect true green
    Gmin = np.array([35, 50, 50])
    Gmax = np.array([90, 255, 255])
    img_Gbin = cv2.inRange(img_hsv, Gmin, Gmax)
    list_mask.append(img_Gbin)
    
    # Detect true orange
    Omin = np.array([11, 43, 46])
    Omax = np.array([35, 255, 255])
    img_Obin = cv2.inRange(img_hsv, Omin, Omax)
    list_mask.append(img_Obin)

    # Détection d'image sombre
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    dark = np.sum(img_gray < 50)
    if dark > 0.3 * img_gray.size:
        # Detect true red
        Rmin = np.array([0, 150, 150])
        Rmax = np.array([15, 240, 240])
        img_Rbin_1 = cv2.inRange(img_hsv, Rmin, Rmax)
        
        Rmin = np.array([156, 150, 150])
        Rmax = np.array([180, 240, 240])
        img_Rbin_2 = cv2.inRange(img_hsv, Rmin, Rmax)
        img_Rbin = np.maximum(img_Rbin_1, img_Rbin_2)
        list_mask.append(img_Rbin)
        
        # Detect true green
        Gmin = np.array([35, 150, 150])
        Gmax = np.array([90, 255, 255])
        img_Gbin = cv2.inRange(img_hsv, Gmin, Gmax)
        list_mask.append(img_Gbin)
        
        # Detect true orange
        Omin = np.array([11, 150, 150])
        Omax = np.array([35, 255, 255])
        img_Obin = cv2.inRange(img_hsv, Omin, Omax)
        list_mask.append(img_Obin)
        
        
    if is_image_too_blue(img):
        Vmin = np.array([147-10, 43, 46])
        Vmax = np.array([147+10, 255, 255])
        img_Vbin = cv2.inRange(img_hsv, Vmin, Vmax)
        list_mask.append(img_Vbin)
        
        Vmin = np.array([128-10, 43, 46])        
        Vmax = np.array([128+10, 255, 255])
        img_Vbin = cv2.inRange(img_hsv, Vmin, Vmax)
        list_mask.append(img_Vbin)
        
        Vmin = np.array([110-10, 43, 46])
        Vmax = np.array([110+10, 255, 240])
        img_Vbin = cv2.inRange(img_hsv, Vmin, Vmax)
        list_mask.append(img_Vbin)
        
    if erode:
        kernel = np.ones((5,5), np.uint8)
        list_mask = [cv2.erode(mask, kernel) for mask in list_mask]
    
    if dilate:
        kernel = np.ones((5,5), np.uint8)        
        list_mask = [cv2.dilate(mask, kernel) for mask in list_mask]
            
    if opening:
        kernel = np.ones((5,5), np.uint8)
        list_mask = [cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel) for mask in list_mask]
        
    if closing:
        kernel = np.ones((5,5), np.uint8)
        list_mask = [cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel) for mask in list_mask]
            
    if both:
        kernel = np.ones((5,5), np.uint8)
        list_mask = [cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel) for mask in list_mask]
        list_mask = [cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel) for mask in list_mask]        

    # On enleve de tous les masques les blobs de moins de 10x10 pixels
    final_list = []
    for mask in list_mask:
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if w < 10 and h < 10:
                mask[y:y+h, x:x+w] = 0
        final_list.append(mask)
        
    # On ne garde que les masks avec plus de 100 pixels blancs
    list_mask = [mask for mask in final_list if np.sum(mask) > 100]
    return list_mask


def get_bouding_rect(mask):
    """
    Get all bouding rectangle in mask avec un padding de 10 pixels
    """
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    boxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w < 10 or h < 10:
            continue
        
        x = max(0, x - 10)
        y = max(0, y - 10)
        w += 20 if max(0, x - 10) != 0 else 10
        h += 20 if max(0, y - 10) != 0 else 10
        
        if x + w > mask.shape[1]:
            w = mask.shape[1] - x
        if y + h > mask.shape[0]:
            h = mask.shape[0] - y
        boxes.append([x, y, x+w, y+h])
        
        # Also three rectangles with h = 3*h
        # 1. rectangle on the bottom
        # 2. rectangle on the top
        # 3. rectangle centered

        h = 3 * h
        
        if y + h > mask.shape[0]:
            h = mask.shape[0] - y        
        
        boxes.append([x, y, x+w, y+h])

        if y - h < 0:
            h = y
        boxes.append([x, y-h, x+w, y])
        
        if y - h//2 < 0:
            h = y
            
        if y + h//2 > mask.shape[0]:
            h = mask.shape[0] - y
        boxes.append([x, y-h//2, x+w, y+h//2])
        
    return boxes

def non_max_suppression(boxes, seuil_iou=0.5, seuil_score=0.5):
    """
    boxes : list of boxes (x1, y1, x2, y2, score, pred)
    seuil : intersection over union threshold
    """    
    final_boxes = []
    boxes = sorted(boxes, key=lambda x: x[4], reverse=True)
    for box in boxes:
        discard = False
        for other_box in boxes:
            if intersection_over_union(box[:4], other_box[:4]) > seuil_iou and box[4] < other_box[4]:
                discard = True
                break
            
        if not discard:
            final_boxes.insert(0, box)
            
    final_boxes = [box for box in final_boxes if box[4] >= seuil_score]
    
    return final_boxes

In [15]:
taille = 32
full_list = {}

for i in tqdm(range(len(test_list))):
    img = os.path.join(test_img, test_list[i])
    img_data = cv2.imread(img)
    img_hog_boxes = []
    cropped_hog_boxes = []
    position_x_y = []
    
    for m in detect_ROI(img, closing=1):
        boxes = get_bouding_rect(m)
        for box in boxes:
            x1, y1, x2, y2 = box
            if x1 == x2 or y1 == y2:
                continue
            
            # Extract image
            cropped = img_data[y1:y2, x1:x2]
            if cropped.shape < (taille, taille, 3):
                cropped = cv2.resize(cropped, (taille, taille), interpolation=cv2.INTER_LANCZOS4)
            elif cropped.shape > (taille, taille, 3):
                cropped = cv2.resize(cropped, (taille, taille), interpolation=cv2.INTER_AREA)
                
            # Save as hog
            hog_img = hog(cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY), orientations=9, pixels_per_cell=(2, 2), visualize=False)
            img_hog_boxes.insert(0, hog_img)
            position_x_y.insert(0, (x1, y1, x2, y2))
            hist_red = cv2.calcHist([cropped], [0], None, [256], [0, 256])
            hist_red = hist_red / hist_red.sum()
            hist_green = cv2.calcHist([cropped], [1], None, [256], [0, 256])
            hist_green = hist_green / hist_green.sum()
            hist_blue = cv2.calcHist([cropped], [2], None, [256], [0, 256])
            hist_blue = hist_blue / hist_blue.sum()
            cropped_hog_boxes.insert(0, np.concatenate((hog_img, hist_red.flatten(), hist_green.flatten(), hist_blue.flatten())))
        
    # Predict
    predictions_detector = model_detector.predict(img_hog_boxes)

    position_x_y = [pos for pos, pred in zip(position_x_y, predictions_detector) if pred == 1]
    cropped_hog_boxes = np.array([box for box, pred in zip(cropped_hog_boxes, predictions_detector) if pred == 1])
            
    # Predict_class
    probabilities = model_classifier.predict_proba(cropped_hog_boxes)
    predictions  = model_classifier.predict(cropped_hog_boxes)
    
    confidence_scores = [probabilities[i][pred] for i, pred in enumerate(predictions)]

    nms_before = [[x1, y1, x2, y2, score, pred] for (x1, y1, x2, y2), score, pred in zip(position_x_y, confidence_scores, predictions) if pred != 0]    
    nms_after = non_max_suppression(nms_before, seuil_iou=0.1, seuil_score=0.20)
    
    # On ecrit dans le fichier de prédiction
    full_list[test_list[i]] = nms_after

100%|██████████| 89/89 [02:15<00:00,  1.52s/it]


In [16]:
# On écrit les prédictions dans les fichiers
v = 0
os.makedirs("predictions", exist_ok=True)
folder = f"predictions//roi_v{v}_test"
os.makedirs(f"{folder}", exist_ok=True)   

for name, v in full_list.items(): 
    with open(f"{folder}//{name.replace('.jpg', '.txt')}", "w") as f:
        for box in v:
            x1, y1, x2, y2, score, pred = box
            f.write(f"{x1} {y1} {x2} {y2} {score} {pred}\n")